# Adding functionality to our pollingplace

So, let's start today by adding some more robustness to our app. For starters, we will be updating detail.html. The code below is what we will need to get started

In [ ]:
<h1>{{ question.question_text }}</h1>

{% if error_message %}<p><strong>{{ error_message }}</strong></p>{% endif %}

<form action="{% url 'polls:vote' question.id %}" method="post">
{% csrf_token %}
{% for choice in question.choice_set.all %}
    <input type="radio" name="choice" id="choice{{ forloop.counter }}" value="{{ choice.id }}" />
    <label for="choice{{ forloop.counter }}">{{ choice.choice_text }}</label><br />
{% endfor %}
<input type="submit" value="Vote" />
</form>

## Let's take a closer look at the code we just wrote:

- Our form displays a radio button for each question choice we currently have in our database. BTW, if you haven't added the choices option to your admin panel...this is the perfect time to do so. The name of each of our radio buttons is called "choice".  This means, when somebody selects one of the radio buttons and submits the form, it’ll send the POST data choice=# where # is the ID of the selected choice. This is the basic concept of HTML forms. The id attached to each radio button comes from our for loop counter which will append a number to the button depending on how many choices we have (1,2,3...n). The value of the button comes directly from our database and is what will be used to display the contents.

- We set the form’s action to {% url 'polls:vote' question.id %}, and we set method="post". Using method="post" (as opposed to method="get") is very important, because the act of submitting this form will alter data server-side. Whenever you create a form that alters data server-side, use method="post". This tip isn’t specific to Django; it’s just good Web development practice.

- forloop.counter indicates how many times the for tag has gone through its loop

- Since we’re creating a POST form (which can have the effect of modifying data), we need to worry about Cross Site Request Forgeries. Thankfully, you don’t have to worry too hard, because Django comes with a very easy-to-use system for protecting against it. In short, all POST forms that are targeted at internal URLs should use the {% csrf_token %} template tag.

### Our next objective is to enable the right to vote!!

Let's take care of this by heading back to our views.py file inside of pollingplace/

Once there, you will replace the current vote function with this code:

In [ ]:
def vote(request, question_id):
    question = get_object_or_404(Question,pk=question_id)
    try:
        seleted_choice = question.choice_set.get(pk=request.POST['choice'])
    except (KeyError, Choice.DoesNotExist):
        #Redisplay the question voting form.
        return render(request, 'polls/detail.html', {
            'question': question,
            'error_message':"You didn't select a choice",
        })
    else:
        seleted_choice.votes += 1
        seleted_choice.save()

        # Always return an HttpResponseRedirect after successfully dealing with POST data.
        # This prevents data from being posted twice if a user hits the back button

        return HttpResponseRedirect(reverse('polls:results', args=(question.id,)))

## Breaking it down..

- request.POST is a dictionary-like object that lets you access submitted data by key name. In this case, request.POST['choice'] returns the ID of the selected choice, as a string. request.POST values are always strings.

- Note that Django also provides request.GET for accessing GET data in the same way – but we’re explicitly using request.POST in our code, to ensure that data is only altered via a POST call.

- request.POST['choice'] will raise KeyError if choice wasn’t provided in POST data. The above code checks for KeyError and redisplays the question form with an error message if choice isn’t given.
    
    - Basically, if we hit the vote button without selecting a choice...we will get an error.

- After incrementing the choice count, the code returns an HttpResponseRedirect rather than a normal HttpResponse. HttpResponseRedirect takes a single argument: the URL to which the user will be redirected (see the following point for how we construct the URL in this case).

- As the Python comment above points out, you should always return an HttpResponseRedirect after successfully dealing with POST data. This tip isn’t specific to Django; it’s just good Web development practice.

- We are using the reverse() function in the HttpResponseRedirect constructor in this example. This function helps avoid having to hardcode a URL in the view function. It is given the name of the view that we want to pass control to and the variable portion of the URL pattern that points to that view. In this case, using the URLconf we set up in Adding-to-pollingplace-app.ipynb, this reverse() call will return a string like

    '/polls/3/results/'
    
 where the 3 is the value of question.id. This redirected URL will then call the 'results' view to display the final page.

### Next we need to update the results function to return relavent results when a user wants to see the results of a question.

In [ ]:
def results(request, question_id):
    question = get_object_or_404(Question, pk=question_id)
    return render(request, 'polls/results.html', {'question': question})

### And now we create the results html

This file is being created in:

    pollingplace/
        templates/
            polls/
                results.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Results</title>
</head>
<body>
    <h1>{{ question.question_text }}</h1>

<ul>
{% for choice in question.choice_set.all %}
    <li>{{ choice.choice_text }} -- {{ choice.votes }} vote{{ choice.votes|pluralize }}</li>
{% endfor %}
</ul>

<a href="{% url 'polls:detail' question.id %}">Vote again?</a>
</body>
</html>

We should now have a working polling app, which admittedly looks very plain...but the main goal here was to get a working app. Mission accomplished!!!

Now, let's make our code look a bit better! We'll do that by using Django's generic views and URLs

Inside of views.py, update your code from what it currently is (a bunch of functions spread everywhere) to a more class based object-oriented approach.

In [ ]:
# Same imports from before but add:

from django.views import generic

#same models import

class IndexView(generic.ListView):
    template_name = 'polls/index.html'
    context_object_name = 'latest_question_list'

    def get_queryset(self):
        """Return the last five published questions."""
        return Question.objects.order_by('-pub_date')[:5]


class DetailView(generic.DetailView):
    model = Question
    template_name = 'polls/detail.html'


class ResultsView(generic.DetailView):
    model = Question
    template_name = 'polls/results.html'

def vote(request, question_id):
    question = get_object_or_404(Question,pk=question_id)
    try:
        seleted_choice = question.choice_set.get(pk=request.POST['choice'])
    except (KeyError, Choice.DoesNotExist):
        #Redisplay the question voting form.
        return render(request, 'polls/detail.html', {
            'question': question,
            'error_message':"You didn't select a choice",
        })
    else:
        seleted_choice.votes += 1
        seleted_choice.save()

        # Always return an HttpResponseRedirect after successfully dealing with POST data.
        # This prevents data from being posted twice if a user hits the back button

        return HttpResponseRedirect(reverse('polls:results', args=(question.id,)))

**urls.py update**

In [ ]:
app_name = 'polls'
urlpatterns = [
    url(r'^$', views.IndexView.as_view(), name='index'),
    url(r'^(?P<pk>[0-9]+)/$', views.DetailView.as_view(), name='detail'),
    url(r'^(?P<pk>[0-9]+)/results/$', views.ResultsView.as_view(), name='results'),
    url(r'^(?P<question_id>[0-9]+)/vote/$', views.vote, name='vote'),
]